#### The objective of this function is to union data from rolling 52 production file with fc_output and export it to rolling 52 production file

In [7]:
import Bens_forecasting_utils as fc
import pandas as pd
import datetime
import numpy as np
import logging
import sys

In [8]:
parameters = dict(
    
    # rolling 52 week production:
    sheet_id                    = '1C9qos9wtf-CI3tmNXcIKnbV4V3f-GDFxa54ThF1pCvI',
    tab                         = 'raw!A1:F'
)

In [9]:
parameters_2= dict(
    
    # wfm forecasting fc_output:
    sheet_id                    = '1VyNaIoJaFiz7wxM9qlEdPIujqTPh8LhyyMNf2YcjXPo',
    tab                         = 'fc_output!A1:F'
)

In [10]:
def rolling_52(date, parameters=parameters, parameters_2=parameters_2):
    
    # creating dataframe df:rolling 52 week production
    # fc_output: WFM Forecasting CW
    
    df = fc.import_gsheet_to_df(parameters['sheet_id'], parameters['tab'])
    fc_output = fc.import_gsheet_to_df(parameters_2['sheet_id'], parameters_2['tab'])
    
    # convert forecast_date column to datetime
    fc_output['forecast_date'] = pd.to_datetime(fc_output['forecast_date']).dt.strftime('%d/%m/%Y')
    
    # changing column names in fc_output
    fc_output.rename(columns={'forecast_date':'ds','language/market/iban':'language'},inplace=True)
    
    # merging rolling 52 week production with fc_output
    data = df.merge(fc_output,how='outer',on=['ds','language','channel'])

    # convert ds column to datetime
    data['ds'] = pd.to_datetime(data['ds'],format='%d/%m/%Y')
    
    # adding date to filter and creating a new column
    compare_date = pd.to_datetime(date,format='%d/%m/%Y')
    data['compare'] = np.where(data['ds']<compare_date,'before','after')
    
    # creating before and after to merge them
    before = data[data['compare']=='before'][['ds','language','channel','yhat','yhat_upper','yhat_lower']]
    after = data[data['compare']=='after'][['ds','language','channel','final_vol_forecast',
                                            'final_vol_forecast_upper_bound','final_vol_forecast_lower_bound']]
    
    # renaming columns before union
    after.rename(columns={'final_vol_forecast':'yhat','final_vol_forecast_upper_bound':'yhat_upper',
             'final_vol_forecast_lower_bound':'yhat_lower'},inplace=True)
    
    # union before and after
    roll = pd.concat([before,after])
    
    # converting to float to keep decimals
    roll['yhat'] = pd.to_numeric(roll['yhat']).round(2)
    roll['yhat_upper'] = pd.to_numeric(roll['yhat_upper']).round(2)
    roll['yhat_lower'] = pd.to_numeric(roll['yhat_lower']).round(2)
    
    # converting to datetime for exporting
    roll['ds'] = roll['ds'].dt.strftime('%Y-%m-%d %H:%M:%S')
    na = roll[roll.isna().any(axis=1)]
    
    roll = roll.sort_values(by=['language','channel','ds'])
    
    # export parameters:
    gsheet_export_params = dict(
    
    df                 = roll,
    gsheet_id          = parameters['sheet_id'],
    gsheet_tab_name    = 'raw',
    include_df_headers = True,
    tab_colour         = (0.0, 0.0, 0.0) #RGB tab colour
        )

    fc.export_df_to_google_sheet(**gsheet_export_params)
    
    return na

### taking the copy of the rolling 52 production file before running the function would be safer!

#### make sure that yhat, yhat_upper, yhat_lower is with 2 decimals in rolling book before running the function

In [11]:
# it includes the date you added in the function as well for your information
# if your dataframe is with n/a you should check first ds, then language, then channel to be the same name
rolling_52(date='07/01/2025') 

,ds,language,channel,yhat,yhat_upper,yhat_lower
